In [ ]:
import pandas

import automol

In [ ]:
# 1. read in the species lists
C67_SPC_PATH = 'mechanisms/01_raw_nuig-c6-7.csv'
PYR_SPC_PATH = 'mechanisms/01_raw_nuig-pyro.csv'

C67_SPC_DF = pandas.read_csv(C67_SPC_PATH, quotechar="'")
PYR_SPC_DF = pandas.read_csv(PYR_SPC_PATH, quotechar="'")

In [ ]:
# 2. expand species stereo (full and non-redundant) for each list
def append_stereo_suffix(name, chi, racem=False):
    """ Append a stereo suffix to a CHEMKIN name, based on its ChI string
    """
    bnd_ste_par_dct = automol.chi.bond_stereo_parities(chi)
    atm_ste_par_dct = automol.chi.atom_stereo_parities(chi)
    
    ste_str = ''
    ste_str += ''.join('E' if p else 'Z' for _, p in sorted(bnd_ste_par_dct.items()))
    ste_str += ''.join('B' if p else 'A' for _, p in sorted(atm_ste_par_dct.items()))
    if automol.chi.is_chiral(chi):
        ste_str += ('r' if racem else
                    '1' if automol.chi.is_inverted_enantiomer(chi) else '0')

    return f'{name}-{ste_str}' if ste_str else name


def expand_species_stereo(df, enant=True):
    """ expand stereo for a species DataFrame
    
        :param df: A DataFrame containing an 'inchi' column
            and a 'name' column.
        :param enant: If True, perform a full expansion
            including all enantiomers. If False, perform a
            non-redundant expansion with only canonical
            enantiomers.
    """
    df = df.set_index('inchi')
    row_dcts = []
    for chi, row in df.iterrows():
        name = row['name']
        
        print(f'Expanding {name} {chi}...')
        ste_chis = automol.chi.expand_stereo(chi, enant=enant)
        for ste_chi in ste_chis:
            row_dct = row.to_dict()
            ste_name = append_stereo_suffix(name, ste_chi, racem=False)
            ste_smi = automol.chi.smiles(ste_chi)
            row_dct.update({
                'inchi': ste_chi,
                'smiles': ste_smi,
                'name': ste_name,
                'orig-name': name,
                'orig-inchi': chi,
            })
            row_dcts.append(row_dct)

            print(f' - {ste_name} = {ste_chi} = {ste_smi}')
            automol.chi.display(ste_chi)
    
    ste_df = pandas.DataFrame.from_records(row_dcts)
    return ste_df

In [ ]:
print("Full species stereoexpansion for NUIG-C6-7")
print("WARNING: RDKit fails to properly visualize some radicals")
FU_EXP_C67_SPC_DF = expand_species_stereo(C67_SPC_DF, enant=True)

FU_EXP_C67_SPC_PATH = 'species/02_species-fu-exp_nuig-c6-7.csv'
FU_EXP_C67_SPC_DF.set_index('name', inplace=True)
FU_EXP_C67_SPC_DF.to_csv(FU_EXP_C67_SPC_PATH, quotechar="'")

In [ ]:
print("Non-redundant species stereoexpansion for NUIG-C6-7")
print("WARNING: RDKit fails to properly visualize some radicals")
NR_EXP_C67_SPC_DF = expand_species_stereo(C67_SPC_DF, enant=False)

NR_EXP_C67_SPC_PATH = 'species/02_species-nr-exp_nuig-c6-7.csv'
NR_EXP_C67_SPC_DF.set_index('name', inplace=True)
NR_EXP_C67_SPC_DF.to_csv(NR_EXP_C67_SPC_PATH, quotechar="'")

In [ ]:
print("Full species stereoexpansion for NUIG-Pyro")
print("WARNING: RDKit fails to properly visualize some radicals")
FU_EXP_PYR_SPC_DF = expand_species_stereo(PYR_SPC_DF, enant=True)

FU_EXP_PYR_SPC_PATH = 'species/02_species-fu-exp_nuig-pyro.csv'
FU_EXP_PYR_SPC_DF.set_index('name', inplace=True)
FU_EXP_PYR_SPC_DF.to_csv(FU_EXP_PYR_SPC_PATH, quotechar="'")

In [ ]:
print("Non-redundant species stereoexpansion for NUIG-Pyro")
print("WARNING: RDKit fails to properly visualize some radicals")
NR_EXP_PYR_SPC_DF = expand_species_stereo(PYR_SPC_DF, enant=False)

NR_EXP_PYR_SPC_PATH = 'species/02_species-nr-exp_nuig-pyro.csv'
NR_EXP_PYR_SPC_DF.set_index('name', inplace=True)
NR_EXP_PYR_SPC_DF.to_csv(NR_EXP_PYR_SPC_PATH, quotechar="'")